Limpieza de datos del dataset de víctimas

In [35]:
import pandas as pd
import numpy as np

df = pd.read_csv("../datasets/raw/siniestros_viales_victimas.csv", low_memory=False)

Exploro datos previo a su limpieza

In [36]:
df.info() #transformar tipo de dato de la fecha, año, edad

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_siniestro                 62076 non-null  object 
 1   fecha_siniestro              62076 non-null  object 
 2   anio_siniestro               62076 non-null  float64
 3   modo_desplazamiento_victima  62076 non-null  object 
 4   sexo_victima                 62076 non-null  object 
 5   edad_victima                 62076 non-null  object 
 6   GRAVEdad_victima             62076 non-null  object 
 7   rol_victima                  62076 non-null  object 
 8   fecha_fallecimiento_victima  610 non-null    object 
dtypes: float64(1), object(8)
memory usage: 72.0+ MB


Elimino columnas irrelevantes

In [37]:
df = df.drop(columns=["GRAVEdad_victima", "fecha_fallecimiento_victima", "anio_siniestro", "rol_victima"])
df

,id_siniestro,fecha_siniestro,modo_desplazamiento_victima,sexo_victima,edad_victima
0,LC-2022-0132915,2022-03-20,AUTO,M,0
1,LC-2022-0054811,2022-02-06,AUTO,F,3
2,LC-2022-0166976,2022-04-04,AUTO,M,4
3,LC-2023-0096195,2023-02-22,AUTO,M,5
4,LC-2023-0096195,2023-02-22,AUTO,M,7
...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN
1048571,NaN,NaN,NaN,NaN,NaN
1048572,NaN,NaN,NaN,NaN,NaN
1048573,NaN,NaN,NaN,NaN,NaN


Busco los NaN existentes por columna

In [38]:
df.isna().sum()

id_siniestro                   986499
fecha_siniestro                986499
modo_desplazamiento_victima    986499
sexo_victima                   986499
edad_victima                   986499
dtype: int64

Elimino duplicados y NaN

In [39]:
df = df.drop_duplicates()
df = df.dropna()
df

,id_siniestro,fecha_siniestro,modo_desplazamiento_victima,sexo_victima,edad_victima
0,LC-2022-0132915,2022-03-20,AUTO,M,0
1,LC-2022-0054811,2022-02-06,AUTO,F,3
2,LC-2022-0166976,2022-04-04,AUTO,M,4
3,LC-2023-0096195,2023-02-22,AUTO,M,5
4,LC-2023-0096195,2023-02-22,AUTO,M,7
...,...,...,...,...,...
62071,HC-2019-0373765,2019-06-14,UTILITARIO,M,49
62072,HC-2021-0242208,2021-05-31,UTILITARIO,M,20
62073,HC-2023-0559525,2023-09-27,UTILITARIO,M,53
62074,HC-2024-0120559,2024-03-03,UTILITARIO,F,44


Normalizo columnas

In [40]:
cols = ["id_siniestro", "modo_desplazamiento_victima", "sexo_victima"]
for col in cols:
    df[col] = df[col].str.strip().str.lower()
df

,id_siniestro,fecha_siniestro,modo_desplazamiento_victima,sexo_victima,edad_victima
0,lc-2022-0132915,2022-03-20,auto,m,0
1,lc-2022-0054811,2022-02-06,auto,f,3
2,lc-2022-0166976,2022-04-04,auto,m,4
3,lc-2023-0096195,2023-02-22,auto,m,5
4,lc-2023-0096195,2023-02-22,auto,m,7
...,...,...,...,...,...
62071,hc-2019-0373765,2019-06-14,utilitario,m,49
62072,hc-2021-0242208,2021-05-31,utilitario,m,20
62073,hc-2023-0559525,2023-09-27,utilitario,m,53
62074,hc-2024-0120559,2024-03-03,utilitario,f,44


Transformo el dtype de las columnas

In [41]:
df["fecha_siniestro"] = pd.to_datetime(df["fecha_siniestro"], errors="coerce")

In [42]:
#creo variables con respecto al año, mes y dia
df["año_victima"] = df["fecha_siniestro"].dt.year
df["dia_victima"] = df["fecha_siniestro"].dt.day_name()
df["mes_victima"] = df["fecha_siniestro"].dt.month_name()

In [43]:
#reemplazo "sd" (sin datos) por NaN para luego poder reemplazar NaN por la edad promedio

df["edad_victima"] = pd.to_numeric(df["edad_victima"], errors="coerce")

edad_prom = df["edad_victima"].mean()

df["edad_victima"] = df["edad_victima"].fillna(edad_prom)


Creo la variable rango_edad

In [44]:
def categorizar_edad(edad):
    if edad <= 12:
        return "0-12"
    elif edad <= 18:
        return "13-18"
    elif edad <= 30:
        return "19-30"
    elif edad <= 45:
        return "31-45"
    elif edad <= 60:
        return "46-60"
    else:
        return "60+"
    
df["rango_edad"] = df["edad_victima"].apply(categorizar_edad)

Reviso si existen valores atípicos

In [45]:
print(df["edad_victima"].min(), df["edad_victima"].max()) #el rango es incorrecto

0.0 100.0


In [46]:
#busco en las filas los valores iguales a 100 y 0 para reemplazarlos por NaN
df.loc[(df["edad_victima"] == 0) | (df["edad_victima"] == 100), "edad_victima"] = np.nan

#reemplazo NaN por edad promedio y formateo el tipo de dato a int nuevamente

df["edad_victima"] = df["edad_victima"].fillna(edad_prom).astype(int)

df["edad_victima"].dtype

dtype('int64')

In [47]:
df["modo_desplazamiento_victima"].unique() 

array(['auto', 'bicicleta', 'camion', 'monopatin', 'moto', 'movil',
       'otro', 'peaton', 'sd', 'taxi', 'transporte publico', 'utilitario'],
      dtype=object)

In [48]:
df.isnull().sum() 

id_siniestro                   0
fecha_siniestro                0
modo_desplazamiento_victima    0
sexo_victima                   0
edad_victima                   0
año_victima                    0
dia_victima                    0
mes_victima                    0
rango_edad                     0
dtype: int64

In [49]:
df.dtypes

id_siniestro                           object
fecha_siniestro                datetime64[ns]
modo_desplazamiento_victima            object
sexo_victima                           object
edad_victima                            int64
año_victima                             int32
dia_victima                            object
mes_victima                            object
rango_edad                             object
dtype: object

Dataframe listo para exportar 

In [50]:
df.to_csv("../datasets/processed/siniestros_viales_victimas_limpio.csv")